In [1]:
from pandas import Timestamp, Timedelta
import emaremes as mrms

**Precipitation rate data**

MRMS data is available every 2 minutes.

In [2]:
preciprate_files = mrms.download.timerange(
    Timestamp("2024-09-27T00:00:00"),
    Timestamp("2024-09-28T00:00:00"),
    frequency=Timedelta(minutes=20),
)


print("First 5 files:")
for f in preciprate_files[:5]:
    print(f)

print("\nLast 5 files:")
for f in preciprate_files[-5:]:
    print(f)

First 5 files:
/home/edwin/emaremes/20240927/PrecipRate_00.00_20240927-000000.grib2.gz
/home/edwin/emaremes/20240927/PrecipRate_00.00_20240927-002000.grib2.gz
/home/edwin/emaremes/20240927/PrecipRate_00.00_20240927-004000.grib2.gz
/home/edwin/emaremes/20240927/PrecipRate_00.00_20240927-010000.grib2.gz
/home/edwin/emaremes/20240927/PrecipRate_00.00_20240927-012000.grib2.gz

Last 5 files:
/home/edwin/emaremes/20240927/PrecipRate_00.00_20240927-224000.grib2.gz
/home/edwin/emaremes/20240927/PrecipRate_00.00_20240927-230000.grib2.gz
/home/edwin/emaremes/20240927/PrecipRate_00.00_20240927-232000.grib2.gz
/home/edwin/emaremes/20240927/PrecipRate_00.00_20240927-234000.grib2.gz
/home/edwin/emaremes/20240928/PrecipRate_00.00_20240928-000000.grib2.gz


-----
**Precipitation flags**

From the [mrms-support repo](https://github.com/NOAA-National-Severe-Storms-Laboratory/mrms-support/blob/3edf7c25f503f81a12eb179dbbd3d4dae607e477/GRIB2_TABLES/UserTable_MRMS_PrecipFlags.csv): 

|Flag|Description|
|:-:|:-|
|-3|no coverage|
|0|no precipitation|
|1|warm stratiform rain|
|3|snow|
|6|convective rain|
|7|rain mixed with hail|
|10|cold stratiform rain|
|91|tropical/stratiform rain mix|
|96|tropical/convective rain mix|

In [3]:
precipflags_files = mrms.download.timerange(
    Timestamp("2024-09-27T00:00:00"),
    Timestamp("2024-09-28T00:00:00"),
    frequency=Timedelta(minutes=20),
    data_type="precip_flag",
)

print("First 5 files:")
for f in precipflags_files[:5]:
    print(f)

print("\nLast 5 files:")
for f in precipflags_files[-5:]:
    print(f)

First 5 files:
/home/edwin/emaremes/20240927/PrecipFlag_00.00_20240927-000000.grib2.gz
/home/edwin/emaremes/20240927/PrecipFlag_00.00_20240927-002000.grib2.gz
/home/edwin/emaremes/20240927/PrecipFlag_00.00_20240927-004000.grib2.gz
/home/edwin/emaremes/20240927/PrecipFlag_00.00_20240927-010000.grib2.gz
/home/edwin/emaremes/20240927/PrecipFlag_00.00_20240927-012000.grib2.gz

Last 5 files:
/home/edwin/emaremes/20240927/PrecipFlag_00.00_20240927-224000.grib2.gz
/home/edwin/emaremes/20240927/PrecipFlag_00.00_20240927-230000.grib2.gz
/home/edwin/emaremes/20240927/PrecipFlag_00.00_20240927-232000.grib2.gz
/home/edwin/emaremes/20240927/PrecipFlag_00.00_20240927-234000.grib2.gz
/home/edwin/emaremes/20240928/PrecipFlag_00.00_20240928-000000.grib2.gz


**Accumulated precipitation**

This data is available every hour. Accumulates are available for 1, 24 and 72 hours periods.

In [4]:
daily_files = mrms.download.timerange(
    Timestamp("2024-09-26T12:00:00"),
    Timestamp("2024-09-29T00:00:00"),
    frequency=Timedelta(days=1),
    data_type="precip_accum_24h",
)

print("We should get three files:")
for f in daily_files:
    print(f)

We should get three files:
/home/edwin/emaremes/20240926/RadarOnly_QPE_24H_00.00_20240926-120000.grib2.gz
/home/edwin/emaremes/20240927/RadarOnly_QPE_24H_00.00_20240927-120000.grib2.gz
/home/edwin/emaremes/20240928/RadarOnly_QPE_24H_00.00_20240928-120000.grib2.gz
